In [ ]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# Then install unsloth
!pip install git+https://github.com/unslothai/unsloth.git


In [ ]:
!pip install " git+https://github.com/huggingface/transformers.git"

In [ ]:
!pip install trl

In [ ]:
!pip install bitsandbytes


In [ ]:
!pip install unsloth_zoo


In [ ]:
from unsloth import FastLanguageModel


In [ ]:
import torch

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments


In [ ]:
from datasets import load_dataset

In [ ]:
max_seq_len =2048

In [ ]:
dataset = load_dataset("imdb",split="train")

In [ ]:
dataset

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(model_name="unsloth/mistral-7b-bnb-4bit",dtype=None,load_in_4bit=True)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj",
        "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,  # ✅ lowercase 'lora_alpha'
    lora_dropout=0, # ✅ lowercase 'lora_dropout'
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407
)


In [ ]:
!pip install xformers

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir="unsloth-test",
        optim="paged_adamw_8bit",
        seed=3407
    ),
)
trainer.train()

In [ ]:
inputs = tokenizer(
    [
        "I really like movie because it shows emotion and Humanity"
    ],
    return_tensors="pt",
).to("cuda")

In [ ]:
inputs

In [ ]:
#Generate output from model
outputs = model.generate(
    **inputs,
    max_new_tokens =128,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id
)
decoded = tokenizer.batch_decode(outputs,skip_speacial_tokens=True)
cleaned =[text.replace("\n","").strip() for text in decoded]

print(claeaned)

In [ ]:
outputs

In [ ]:
model.save_pretrained("lora_model")